<a href="https://colab.research.google.com/github/ParvezAlam-AI/ai-residency-aug24/blob/main/stock_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 665.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00


In [3]:
!pip install yfinance


In [4]:
from google.colab import userdata

In [5]:
import pandas as pd
import openai
import gradio as gr
import requests
import yfinance as yf

In [6]:
symbol = 'TSLA'

stock = yf.Ticker(symbol)
price = stock.history(period='1d')

latest_price = price['Close'][0]

print(f'The price of {stock} is {latest_price}')




The price of yfinance.Ticker object <TSLA> is 256.2300109863281


<ipython-input-6-f513812bc7ee>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_price = price['Close'][0]


In [7]:
def stock_price(ticker):
  stock = yf.Ticker(ticker)
  price = stock.history(period='1d')
  latest_price = price['Close'].iloc[0]
  cashflow = stock.cashflow

  return latest_price,cashflow

In [8]:
stock_price('TSLA')

(256.2300109863281,
                                                    2023-12-31     2022-12-31  \
 Free Cash Flow                                   4357000000.0   7552000000.0   
 Repayment Of Debt                               -1815000000.0  -3866000000.0   
 Issuance Of Debt                                 3931000000.0            0.0   
 Issuance Of Capital Stock                                 NaN            0.0   
 Capital Expenditure                             -8899000000.0  -7172000000.0   
 Interest Paid Supplemental Data                   126000000.0    152000000.0   
 Income Tax Paid Supplemental Data                1119000000.0   1203000000.0   
 End Cash Position                               17189000000.0  16924000000.0   
 Beginning Cash Position                         16924000000.0  18144000000.0   
 Effect Of Exchange Rate Changes                     4000000.0   -444000000.0   
 Changes In Cash                                   261000000.0   -776000000.0   
 Financi

#function

In [9]:
functions =[
     {
        "name": "stock_price",
        "description": "Get the current stock price",
        "strict": True,
        "parameters": {
          "type": "object",
          "properties": {
            "symbol": {
              "type": "string",
              "description": "The stock symbol"
            }
          },
          "additionalProperties": False,
          "required": [
            "symbol"
          ]
        }
      }
]

In [10]:
functions

[{'name': 'stock_price',
  'description': 'Get the current stock price',
  'strict': True,
  'parameters': {'type': 'object',
   'properties': {'symbol': {'type': 'string',
     'description': 'The stock symbol'}},
   'additionalProperties': False,
   'required': ['symbol']}}]

In [11]:
openai.api_key = userdata.get('apopen')

In [18]:
#function will be used in the below code. structure is copied from the documentation.
def stock_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a stock information bot. Answer only related to stock information."})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=50,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )



    #Handling Function Calls and Fetching stocks Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])
        # arguments = json.loads(function_call['arguments'])

        # Fetch stocks data using the extracted arguments
        stock_data = stock_price(arguments['symbol'])

        # Append the function call and stocks data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "stock_price", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "stock_price", "content": str(stock_data)})

        #magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide stocks information. Please ask me questions related to Stock Market."

In [21]:
stock_chat('what is the CASHFLOW AND OTHER FINANCIALS of tsla today')

'As of the most recent financial data available, Tesla (TSLA) reported the following financials:\n\n- Free Cash Flow: $4.35 billion\n- Repayment of Debt: -$1.815 billion\n- Issuance of Debt: $3.931 billion\n- Capital Expenditure: -$8.899 billion\n- Interest Paid: $126 million\n- Income Tax Paid: $1.119 billion\n- Operating Cash Flow: $13.256 billion\n\nPlease note that financial data can vary over time and it is recommended to refer to the latest financial reports for the most up-to-date information.'

In [14]:
# import openai
# Make sure you have the openai package installed. If not, use !pip install openai
# !pip install openai

# Set your OpenAI API key here. Replace <API-KEY> with your actual key
openai.api_key = userdata.get('apopen')

#function will be used in the below code. structure is copied from the documentation.
def stock_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a stock information bot. Answer only related to stock information."})


        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )



    #Handling Function Calls and Fetching stocks Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch stocks data using the extracted arguments
        # Note: This assumes you have a function named 'stock_price' defined elsewhere
        stock_data = stock_price(arguments['ticker'])

        # Append the function call and stocks data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "stock_price", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "stock_price", "content": str(stock_data)})

        #magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide stocks information. Please ask me questions related to Stock Market."

In [15]:
stock_chat("what is the status of TSLA today")

InvalidRequestError: Invalid function definition for 'stock_price': the 'strict' parameter is only permitted in 'tools'.

In [16]:
# !pip install openai==0.28
# !pip install gradio
# !pip install yfinance

from google.colab import userdata
import pandas as pd
import openai
import gradio as gr
import requests
import yfinance as yf
import json

symbol = 'TSLA'

# Fetch latest stock price
def stock_price(ticker):
    stock = yf.Ticker(ticker)
    price = stock.history(period='1d')
    latest_price = price['Close'].iloc[0]
    cashflow = stock.cashflow
    return latest_price, cashflow

# Define function schema
functions = [
    {
        "name": "stock_price",
        "description": "Get the current stock price",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol"
                }
            },
            "additionalProperties": False,
            "required": [
                "symbol"
            ]
        }
    }
]

openai.api_key = userdata.get('apopen')

# Main function to interact with OpenAI API
def stock_chat(user_message):
    messages = [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": "You are a stock information bot. Answer only related to stock information."}
    ]

    # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=50,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        messages=messages,
        functions=functions
    )

    # Handling Function Calls and Fetching Stocks Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = json.loads(function_call['arguments'])

        # Fetch stock data using the extracted arguments
        stock_data = stock_price(arguments['symbol'])

        # Append the function call and stocks data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_stock_price", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_stock_price", "content": str(stock_data)})

        # Magic of LLM
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "I'm here to provide stocks information. Please ask me questions related to the Stock Market."




In [17]:
# Example call
print(stock_chat("give me the history of TSLA?"))

Tesla, Inc. (TSLA) is an American electric vehicle and clean energy company founded in 2003 by Martin Eberhard and Marc Tarpenning. The company's current CEO, Elon Musk, became involved in Tesla shortly after its founding and has been instrumental in its growth and success.

Tesla's mission is to accelerate the transition to sustainable energy by producing electric vehicles, solar products, and energy storage solutions. The company's electric vehicles have gained popularity for their innovative design, performance, and environmental benefits.

Since its founding, Tesla has experienced significant growth and has become one of the leading electric vehicle manufacturers in the world. The company went public in 2010 and has seen its stock price rise significantly in recent years, making it one of the most valuable automakers in the world.

Tesla continues to innovate in the electric vehicle and renewable energy space, with plans to expand its product line and production capacity in the com